In [ ]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

#this is a constant needed for stacking (not hyperparameter)
market_train_df = market_train_df.loc[market_train_df['time'] >= '2010-01-01 22:00:00+0000']

<h1>Basic data cleaning

In [3]:
import numpy as np
import pandas as pd
#from tqdm import tqdm_notebook as tqdm
from time import time
#from matplotlib import pyplot as plt

In [18]:
from StringIO import StringIO

In [19]:
h=StringIO("""',time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe\n1126217,2010-01-04 22:00:00+00:00,A.N,Agilent Technologies Inc,2729240.0,31.3,31.39,0.007402639201802952,0.011275773195873029,-0.0023437029039039773,0.010532228852100629,0.06317934782608936,0.06587436332767194,0.027269592381124926,0.06009231917386074,-0.040196846038035944,1.0\n1126218,2010-01-04 22:00:00+00:00,AAI.N,AirTran Holdings Inc,3436803.0,5.18,5.26,-0.007662835249042099,0.011538461538461497,-0.027851042066156708,0.021130641959079773,0.005825242718446422,0.005736137667303964,0.08355775694246992,0.03017750557429531,0.06125265695209382,0.0\n1126219,2010-01-04 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1701655.0,40.38,40.7,-0.0024703557312254487,-0.009973242520068482,-0.012789111603922616,-0.008616765940437202,-0.008377817529279019,0.0026768764797466815,-0.0217879114560604,-0.0010338609902458464,-0.03412597268689382,1.0\n1126220,2010-01-04 22:00:00+00:00,AAPL.O,Apple Inc,17633150.0,214.01,213.5,0.015555302469489662,0.0018300408239853017,-0.0013293217047078594,0.009181640302293733,0.11544876472427745,0.09560219633601652,0.011769112745131247,0.06708613524655585,-0.03582750494004208,1.0\n1126221,2010-01-04 22:00:00+00:00,AAV.N,Advantage Oil & Gas Ltd,833228.0,6.83,6.61,0.04754601226993871,-0.003016591251885359,0.028141249330657255,-0.0006611631351533314,0.05727554179566563,0.03767660910518056,0.002492374061736108,0.01198910323360472,0.005125413526593476,0.0\n1126222,2010-01-04 22:00:00+00:00,AAWW.O,Atlas Air Worldwide Holdings Inc,607417.0,38.2,37.82,0.025503355704697972,0.007995735607675858,0.022891816013946083,-0.0005192480928454188,0.027710519235943032,0.028276237085372413,0.08467565935012888,0.04448254462034561,0.045759341886862426,0.0\n1126223,2010-01-04 22:00:00+00:00,AB.N,AllianceBernstein Holding LP,556791.0,28.4,28.51,0.010676156583624863,0.011710432931155701,-0.007549100742243985,0.014918861557563005,0.08979278587873663,0.06380597014925571,0.017159687788576045,0.05117530070714556,0.014543055530893068,1.0\n1126224,2010-01-04 22:00:00+00:00,ABB.N,ABB Ltd,1847102.0,19.64,19.59,0.028272251308903318,0.015552099533437058,0.008189690330353348,0.02037003779861471,0.08269018743109213,0.07049180327869231,-0.022457902390133858,0.04128849051754045,-0.05344358487171035,1.0\n1126225,2010-01-04 22:00:00+00:00,ABC.N,AmerisourceBergen Corp,2455833.0,26.63,26.29,0.021480629075565805,-0.0037893141341412973,0.007690742559164502,0.0019827365220369507,0.029775715390566226,0.006893910379163426,-0.011077076597308319,-0.007055307926889116,0.03353088338610885,1.0\n1126226,2010-01-04 22:00:00+00:00,ABD.N,ACCO Brands Corp,389300.0,7.9,7.49,0.08516483516483464,-0.006631299734748186,0.0453725499345708,-0.004467637426902099,0.07191316146540006,0.002677376171351886,0.07958208991202102,0.010347732398656287,0.06283915783793581,0.0\n'""")

In [20]:
pd.read_csv(h)

,',time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe
0,1126217,2010-01-04 22:00:00+00:00,A.N,Agilent Technologies Inc,2729240.0,31.30,31.39,0.007403,0.011276,-0.002344,0.010532,0.063179,0.065874,0.027270,0.060092,-0.040197,1.0
1,1126218,2010-01-04 22:00:00+00:00,AAI.N,AirTran Holdings Inc,3436803.0,5.18,5.26,-0.007663,0.011538,-0.027851,0.021131,0.005825,0.005736,0.083558,0.030178,0.061253,0.0
2,1126219,2010-01-04 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1701655.0,40.38,40.70,-0.002470,-0.009973,-0.012789,-0.008617,-0.008378,0.002677,-0.021788,-0.001034,-0.034126,1.0
3,1126220,2010-01-04 22:00:00+00:00,AAPL.O,Apple Inc,17633150.0,214.01,213.50,0.015555,0.001830,-0.001329,0.009182,0.115449,0.095602,0.011769,0.067086,-0.035828,1.0
4,1126221,2010-01-04 22:00:00+00:00,AAV.N,Advantage Oil & Gas Ltd,833228.0,6.83,6.61,0.047546,-0.003017,0.028141,-0.000661,0.057276,0.037677,0.002492,0.011989,0.005125,0.0
5,1126222,2010-01-04 22:00:00+00:00,AAWW.O,Atlas Air Worldwide Holdings Inc,607417.0,38.20,37.82,0.025503,0.007996,0.022892,-0.000519,0.027711,0.028276,0.084676,0.044483,0.045759,0.0
6,1126223,2010-01-04 22:00:00+00:00,AB.N,AllianceBernstein Holding LP,556791.0,28.40,28.51,0.010676,0.011710,-0.007549,0.014919,0.089793,0.063806,0.017160,0.051175,0.014543,1.0
7,1126224,2010-01-04 22:00:00+00:00,ABB.N,ABB Ltd,1847102.0,19.64,19.59,0.028272,0.015552,0.008190,0.020370,0.082690,0.070492,-0.022458,0.041288,-0.053444,1.0
8,1126225,2010-01-04 22:00:00+00:00,ABC.N,AmerisourceBergen Corp,2455833.0,26.63,26.29,0.021481,-0.003789,0.007691,0.001983,0.029776,0.006894,-0.011077,-0.007055,0.033531,1.0
9,1126226,2010-01-04 22:00:00+00:00,ABD.N,ACCO Brands Corp,389300.0,7.90,7.49,0.085165,-0.006631,0.045373,-0.004468,0.071913,0.002677,0.079582,0.010348,0.062839,0.0


In [ ]:
def prepare_data(market_obs_df):

    """
    baseline data cleaning procedure
    Args:
        market_obs_df (market_train_df): from env.get_training_data()
        
    didn't check for N.A.s
    """
    start_time = time()
    
    market_obs_df.reset_index(drop=True, inplace=True)
    
    market_obs_df['close_to_open'] =  np.abs(market_obs_df['close'] / market_obs_df['open'])
    market_obs_df['assetName_mean_open'] = market_obs_df.groupby('assetName')['open'].transform('mean')
    market_obs_df['assetName_mean_close'] = market_obs_df.groupby('assetName')['close'].transform('mean')

    # if open price is too far from mean open price for this company, replace it. Otherwise replace close price.
    for i, row in market_obs_df.loc[market_obs_df['close_to_open'] >= 2].iterrows():
        if np.abs(row['assetName_mean_open'] - row['open']) > np.abs(row['assetName_mean_close'] - row['close']):
            market_obs_df.iloc[i,5] = row['assetName_mean_open']
        else:
            market_obs_df.iloc[i,4] = row['assetName_mean_close']

    for i, row in market_obs_df.loc[market_obs_df['close_to_open'] <= 0.5].iterrows():
        if np.abs(row['assetName_mean_open'] - row['open']) > np.abs(row['assetName_mean_close'] - row['close']):
            market_obs_df.iloc[i,5] = row['assetName_mean_open']
        else:
            market_obs_df.iloc[i,4] = row['assetName_mean_close']
    
    print("TIME: %.2f for cleaning data" % (time()-start_time))
    

In [ ]:
# sanity check on prepare data
X = market_train_df.copy()
prepare_data(X)
assert len(market_train_df.columns) != len(X.columns)
del X

TODO: if I try to run linear regressor it gives me You might need a way of handling missing values, such as pandas.DataFrame.fillna or sklearn.preprocessing.Imputer. See our Missing Values tutorial for more details.

-----

<h1>Models

<h1>Linear regressor

In [ ]:
def model_lr(X, Y):
    """
    this is the core model used for final predictions
    Args:
        X: pandas.df
        Y: values
    Return:
        model, results (None)
    """
    
    prepare_data(X) 
    Y = Y.clip(Y.quantile(0.001), Y.quantile(0.999))
    Y.reset_index(drop=True, inplace=True)
    
    X = X.iloc[:, (X.columns != 'assetCode') 
               & (X.columns != 'assetName') 
               & (X.columns != 'time') 
               & (X.columns != 'returnsOpenNextMktres10')
               & (X.columns != 'period')]
    X, Y = X.fillna(0), Y.fillna(0)
    
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression()
    lr.fit(X.values, Y)
    
    del X, Y
    return lr, None

def linear_regressor(lr, X_test):
    """simple lr
    Args:
        X: pandas.df
        Y: values
        X_test: not values
    Return:
        predictions
    """
    
    prepare_data(X_test) 
    
    X_test = X_test.iloc[:, (X_test.columns != 'assetCode') 
               & (X_test.columns != 'assetName') 
               & (X_test.columns != 'time') 
               & (X_test.columns != 'returnsOpenNextMktres10')
               & (X_test.columns != 'period')]
    X_test = X_test.fillna(0) #fillna must be in values?
    res = lr.predict(X_test.values).clip(-1, 1)
    
    del X_test
    return res

<h2>Example
<p>This is how to use the model for stacking, with linear_regressor

In [ ]:
X, Y = market_train_df.drop('returnsOpenNextMktres10',axis=1), market_train_df['returnsOpenNextMktres10']
split = int(len(X) * 0.8)
X_train, Y_train = X[:split], Y[:split]
X_test, Y_test = X[split:], Y[split:]

lr, _ = model_lr(X_train.copy(), Y_train.copy())
pred_lr = linear_regressor(lr, X_test.copy()) #DEBUG: important to use copy!

from sklearn.metrics import r2_score
print('Test sigma score for linreg in block %d is %f' % (0, r2_score(Y_test.values, pred_lr)))

<h1>Full model predictions
<p>This is the model to make predictions after stacking

In [ ]:
X, Y = market_train_df, market_train_df['returnsOpenNextMktres10']
model00, training_results00 = model_lr(X.copy(), Y.copy())

In [ ]:
type(model00)

<h2>LGB_0629

In [ ]:
def model_lgb_0629(X, Y):
        """
        this is the lightLGB_0629 core model that returns model (used for real predictions)
        Args:
            X: features
            Y: label
        Returns:
            model: lightgbm instance
            training_results: dict with training results
        """
        import lightgbm as lgb
        start_time = time()
        
        prepare_data(X) 
        Y = Y.clip(Y.quantile(0.001), Y.quantile(0.999))
        Y.reset_index(drop=True, inplace=True)
        
        def sigma_score(preds, valid_data):
            """metric definition"""
            df_time = valid_data.params['extra_time'] # will be injected afterwards
            labels = valid_data.get_label()

        #    assert len(labels) == len(df_time)

            x_t = preds * labels #  * df_valid['universe'] -> Here we take out the 'universe' term because we already keep only those equals to 1.

            # Here we take advantage of the fact that `labels` (used to calculate `x_t`)
            # is a pd.Series and call `group_by`
            x_t_sum = x_t.groupby(df_time).sum()
            score = x_t_sum.mean() / x_t_sum.std()

            return 'sigma_score', score, True
        
        #split train validation
        split = int(len(X) * 0.8)
        test_train_distsance = 2000
        X_train, X_val = X[:split - test_train_distsance], X[split:]
        Y_train, Y_val = Y[:split - test_train_distsance], Y[split:]

        #take out universe = 0 from validation
        universe_filter = X['universe'][split:] == 1.0
        X_val = X_val[universe_filter]
        Y_val = Y_val[universe_filter]

        # this is a time_val series used to calc the sigma_score later, applied split and universe filter
        time_val = X['time'][split:][universe_filter]
        assert len(time_val) == len(X_val)
        time_train = X['time'][:split - test_train_distsance]
        assert len(time_train) == len(X_train)

        X_train = X_train.iloc[:, (X_train.columns != 'assetCode') 
                   & (X.columns != 'assetName') 
                   & (X.columns != 'time') 
                   & (X.columns != 'returnsOpenNextMktres10')
                   & (X.columns != 'period')]

        X_val = X_val.iloc[:, (X_val.columns != 'assetCode') 
                   & (X.columns != 'assetName') 
                   & (X.columns != 'time') 
                   & (X.columns != 'returnsOpenNextMktres10')
                   & (X.columns != 'period')]

        assert len(X_train.columns) == len(X_val.columns)

        train_cols = X_train.columns.tolist()

        lgb_train = lgb.Dataset(X_train.values, Y_train, feature_name=train_cols, free_raw_data=False)
        lgb_val = lgb.Dataset(X_val.values, Y_val, feature_name=train_cols, free_raw_data=False)

        lgb_train.params = {
            'extra_time' : time_train.factorize()[0]
        }
        lgb_val.params = {
            'extra_time' : time_val.factorize()[0]
        }

        x_1 = [0.19000424246380565, 2452, 212, 328, 202]
        #this is from eda script 67
        lgb_params = {
                'task': 'train',
                'boosting_type': 'gbdt',
                'objective': 'regression_l1',
        #         'objective': 'regression',
                'learning_rate': x_1[0],
                'num_leaves': x_1[1],
                'min_data_in_leaf': x_1[2],
        #         'num_iteration': x_1[3],
                'num_iteration': 239,
                'max_bin': x_1[4],
                'verbose': 1,
                'lambda_l1': 0.0,
                'lambda_l2' : 1.0,
                'metric':'None'
        }

        training_results = {}
        model = lgb.train(lgb_params, lgb_train, num_boost_round=1000, valid_sets=(lgb_val,lgb_train), valid_names=('valid','train'), verbose_eval=25,
                      early_stopping_rounds=20, feval=sigma_score, evals_result=training_results)

        print("\n\nTIME - lgb_0629: {}".format(time()-start_time))
        
        del X, Y, X_train, Y_train, X_val, Y_val
        return model, training_results

    
def lgb_0629(model, X_test):
    """
    this is the lightLGB model that got 0.629 scores on public LB (with parameters from script 67)
    Args:
        lgb: lgb model
        X_test: features (not values)
    Returns:
        prediction: Y_test
    """
    
    prepare_data(X_test)
    
    X_test = X_test.iloc[:, (X_test.columns != 'assetCode') 
               & (X_test.columns != 'assetName') 
               & (X_test.columns != 'time') 
               & (X_test.columns != 'returnsOpenNextMktres10')
               & (X_test.columns != 'period')]
    
    res = model.predict(X_test.values).clip(-1, 1)
    
    del X_test
    return res

<h2>Example
<p>This is how to use the model for stacking, with lgb_0629 function

In [ ]:
X, Y = market_train_df, market_train_df['returnsOpenNextMktres10']

split = int(len(X) * 0.8)
X_train, Y_train = X[:split], Y[:split]
X_test, Y_test = X[split:], Y[split:]

model, results = model_lgb_0629(X_train.copy(), Y_train.copy())
predictions = lgb_0629(model, X_test.copy()) #DEBUG: did you use copy?

from sklearn.metrics import r2_score
print('Test sigma score for lbg in block %d is %f' % (0, r2_score(Y_test.values, predictions)))

plt.figure(figsize=(8,4))
plt.plot(results['train']['sigma_score'])
plt.plot(results['valid']['sigma_score'])

<h2>Full-model predictions
<p>This is the model to make predictions after stacking

In [ ]:
X, Y = market_train_df, market_train_df['returnsOpenNextMktres10']
model01, training_results01 = model_lgb_0629(X.copy(), Y.copy())

plt.figure(figsize=(8,4))
plt.plot(training_results01['train']['sigma_score'])
plt.plot(training_results01['valid']['sigma_score'])

<h1>First level models stacking

Validation in presence of time component

**f) KFold scheme in time series
**
In time-series task we usually have a fixed period of time we are asked to predict. Like day, week, month or arbitrary period with duration of T.

Split the train data into chunks of duration T. Select first M chunks.
Fit N diverse models on those M chunks and predict for the chunk M+1. Then fit those models on first M+1 chunks and predict for chunk M+2 and so on, until you hit the end. After that use all train data to fit models and get predictions for test. Now we will have meta-features for the chunks starting from number M+1 as well as meta-features for the test.
Now we can use meta-features from first K chunks [M+1,M+2,..,M+K] to fit level 2 models and validate them on chunk M+K+1. Essentially we are back to step 1. with the lesser amount of chunks and meta-features instead of features.

T = 6 month
* 14 periods in Train
* 3 periods in LB
* 1 period in pLB

Build meta features on train periods: [8, 9, 10, 11, 12, 13]

<h2>Building Trainining meta-features

Let's first add periods so that data manipulation will be easier

In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
market_train_df = pd.read_csv("../data/market_train_df.csv").drop('Unnamed: 0', axis=1)

In [2]:
market_train_df = market_train_df.loc[market_train_df['time'] >= '2010-01-01 22:00:00+0000']

periods = ['2010-01-01 22:00:00+0000',
           '2010-06-15 22:00:00+0000',
           '2011-01-01 22:00:00+0000',
           '2011-06-15 22:00:00+0000',
           '2012-01-01 22:00:00+0000',
           '2012-06-15 22:00:00+0000',
           '2013-01-01 22:00:00+0000',
           '2013-06-15 22:00:00+0000',
           '2014-01-01 22:00:00+0000',
           '2014-06-15 22:00:00+0000',
           '2015-01-01 22:00:00+0000',
           '2015-06-15 22:00:00+0000',
           '2016-01-01 22:00:00+0000',
           '2016-06-15 22:00:00+0000',
           '2017-01-01 22:00:00+0000']
market_train_df['period'] = -1
for i, period in enumerate(periods[:-1]):
    market_train_df.loc[(market_train_df['time'] < periods[i + 1]) & (period <= market_train_df['time']), ['period']] = i
cols = market_train_df.columns.tolist()
cols.insert(1, cols[-1])
cols.pop()

# what we did here we created a new column period to easily move 
# between fold, then we prepare level_1 features
X_train_level1 = market_train_df[cols]
y_train_level1 = X_train_level1['returnsOpenNextMktres10']
X_train_level1 = X_train_level1.drop('returnsOpenNextMktres10',axis=1)

# this break if you market_train_df doesn't goe from 2010-01-01 to 2017-06-31
assert len(X_train_level1['period'].unique()) == 14

Now that we cleaned the Database let's build x_train_level2

In [3]:
level_2_range = (8, 14)

# here we make the important choice
# of how long we want to make the stacking for level 2
# in case our predict method is extremely slow
# we might want to decrease level_2_range size

periods_level2 = X_train_level1['period'][X_train_level1['period'].isin(range(level_2_range[0], level_2_range[1]))]
y_train_level2 = y_train_level1[X_train_level1['period'].isin(range(level_2_range[0], level_2_range[1]))]

level1_models = 1 # how many level1 models do we have?
X_train_level2 = np.zeros([y_train_level2.shape[0], level1_models])

In [4]:
X_train_level1.shape, y_train_level1.shape, X_train_level2.shape[1], y_train_level2.shape, periods_level2.shape

((2846738, 17), (2846738,), 1, (1194012,), (1194012,))

In [5]:
X_train_level1

,Unnamed: 0,period,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,universe
1126217,1126217,0,2010-01-04 22:00:00+00:00,A.N,Agilent Technologies Inc,2729240.0,31.30,31.390000,0.007403,0.011276,-0.002344,0.010532,0.063179,0.065874,0.027270,0.060092,1.0
1126218,1126218,0,2010-01-04 22:00:00+00:00,AAI.N,AirTran Holdings Inc,3436803.0,5.18,5.260000,-0.007663,0.011538,-0.027851,0.021131,0.005825,0.005736,0.083558,0.030178,0.0
1126219,1126219,0,2010-01-04 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1701655.0,40.38,40.700000,-0.002470,-0.009973,-0.012789,-0.008617,-0.008378,0.002677,-0.021788,-0.001034,1.0
1126220,1126220,0,2010-01-04 22:00:00+00:00,AAPL.O,Apple Inc,17633150.0,214.01,213.500000,0.015555,0.001830,-0.001329,0.009182,0.115449,0.095602,0.011769,0.067086,1.0
1126221,1126221,0,2010-01-04 22:00:00+00:00,AAV.N,Advantage Oil & Gas Ltd,833228.0,6.83,6.610000,0.047546,-0.003017,0.028141,-0.000661,0.057276,0.037677,0.002492,0.011989,0.0
1126222,1126222,0,2010-01-04 22:00:00+00:00,AAWW.O,Atlas Air Worldwide Holdings Inc,607417.0,38.20,37.820000,0.025503,0.007996,0.022892,-0.000519,0.027711,0.028276,0.084676,0.044483,0.0
1126223,1126223,0,2010-01-04 22:00:00+00:00,AB.N,AllianceBernstein Holding LP,556791.0,28.40,28.510000,0.010676,0.011710,-0.007549,0.014919,0.089793,0.063806,0.017160,0.051175,1.0
1126224,1126224,0,2010-01-04 22:00:00+00:00,ABB.N,ABB Ltd,1847102.0,19.64,19.590000,0.028272,0.015552,0.008190,0.020370,0.082690,0.070492,-0.022458,0.041288,1.0
1126225,1126225,0,2010-01-04 22:00:00+00:00,ABC.N,AmerisourceBergen Corp,2455833.0,26.63,26.290000,0.021481,-0.003789,0.007691,0.001983,0.029776,0.006894,-0.011077,-0.007055,1.0
1126226,1126226,0,2010-01-04 22:00:00+00:00,ABD.N,ACCO Brands Corp,389300.0,7.90,7.490000,0.085165,-0.006631,0.045373,-0.004468,0.071913,0.002677,0.079582,0.010348,0.0


In [ ]:
# Now fill `X_train_level2` with metafeatures
from sklearn.metrics import r2_score
for cur_block_num in range(level_2_range[0], level_2_range[1]):
    print("#"*30)
    print("[STACKING] starting block number "+str(cur_block_num))
    print("#"*30)
    
    
    '''
    [from coursera]
        1. Split `X_train` into parts
           Remember, that corresponding dates are stored in `dates_train` 
        2. Fit linear regression 
        3. Fit LightGBM and put predictions          
        4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
           You can use `dates_train_level2` for it
           Make sure the order of the meta-features is the same as in `X_test_level2`
    '''      
    cur_block_X = X_train_level1[X_train_level1['period'] < cur_block_num]
    cur_block_Y = y_train_level1[X_train_level1['period'] < cur_block_num]
    
    cur_block_X_test = X_train_level1[X_train_level1['period'] == cur_block_num]
    cur_block_Y_test = y_train_level1[X_train_level1['period'] == cur_block_num]
    
    # TODO:   there is NA in values
    # UPDATE: this should be handled by ._generate_features
    
    # let's train here all the models
    '''
    MODEL 1
    : linear regression
    '''
    block_model00, results = model_lr(cur_block_X.copy(), cur_block_Y.copy())
    pred_lr = linear_regressor(block_model00, cur_block_X_test.copy())
    print('Test r2 score for linreg in block %d is %f' % (cur_block_num, r2_score(cur_block_Y_test, pred_lr)))
    
    
    '''
    MODEL 2
    lightLGB (lgb_0629)
    using 'script 67' params
    '''
    block_model01, results = model_lgb_0629(cur_block_X.copy(), cur_block_Y.copy())
    pred_lgb = lgb_0629(block_model01, cur_block_X_test.copy())
    print('Test r2 score for lgb_0629 in block %d is %f' % (cur_block_num, r2_score(cur_block_Y_test, pred_lgb)))
    
    cur_block_X_train_level2 = np.c_[pred_lr, pred_lgb] 
    
    X_train_level2[periods_level2 == cur_block_num] = cur_block_X_train_level2

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(X_train_level2[periods_level2 == 9][:,0].clip(-0.04, 0.04), X_train_level2[periods_level2 == 9][:,1].clip(-0.04, 0.04),alpha=0.3) 
plt.scatter(X_train_level2[periods_level2 == 8][:,0].clip(-0.04, 0.04), X_train_level2[periods_level2 == 8][:,1].clip(-0.04, 0.04),alpha=0.3) 
plt.scatter(X_train_level2[periods_level2 == 10][:,0].clip(-0.04, 0.04), X_train_level2[periods_level2 == 10][:,1].clip(-0.04, 0.04),alpha=0.3) 
plt.scatter(X_train_level2[periods_level2 == 11][:,0].clip(-0.04, 0.04), X_train_level2[periods_level2 == 11][:,1].clip(-0.04, 0.04),alpha=0.3) 
plt.scatter(X_train_level2[periods_level2 == 12][:,0].clip(-0.04, 0.04), X_train_level2[periods_level2 == 12][:,1].clip(-0.04, 0.04),alpha=0.3) 
plt.scatter(X_train_level2[periods_level2 == 13][:,0].clip(-0.04, 0.04), X_train_level2[periods_level2 == 13][:,1].clip(-0.04, 0.04),alpha=0.3) 
plt.title("first level predictions")
plt.xlabel("Predictions of model 0")
plt.ylabel("Predictions of model 1")
plt.plot([-0.04, 0.04], [-0.04, 0.04])

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(results['train']['sigma_score'])
plt.plot(results['valid']['sigma_score'])
#notably good!?

<h1>Stacking</h1>
now let's move to level2, and let's build a model that stacks the predictions of model of level1

In [ ]:
#sanity check on level 2 training test dimensions
assert X_train_level2.shape[0] == np.array(y_train_level2).shape[0]

In [ ]:
from sklearn.linear_model import LinearRegression
level2_model = LinearRegression()

In [ ]:
level2_model.fit(X_train_level2, y_train_level2)

## Main Loop
Let's loop through all the days and make our random predictions.  The `days` generator (returned from `get_prediction_days`) will simply stop returning values once you've reached the end.

In [ ]:
days = env.get_prediction_days()
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    
    market_obs_df['universe'] = 1
    
    """
    MODEL 00 PREDICTIONS
    """
    pred_model00 = linear_regressor(model00, market_obs_df.copy())
    
    """
    MODEL 01 PREDICTIONS
    """
    pred_model01 = lgb_0629(model01, market_obs_df.copy())
    
    """
    META-MODEL PREDICTIONS
    """
    features_level2 = np.c_[pred_model00, pred_model01] 
    
    #with stacking
    #predictions_template_df.confidenceValue = level2_model.predict(features_level2).clip(-1 , 1)
    
    #without stacking
    predictions_template_df.confidenceValue = pred_model01 
    
    
    env.predict(predictions_template_df)
    
print('Done!')

## **`write_submission_file`** function

Writes your predictions to a CSV file (`submission.csv`) in the current working directory.

In [ ]:
env.write_submission_file()

In [ ]:
# We've got a submission file!
import os
print([filename for filename in os.listdir('.') if '.csv' in filename])

As indicated by the helper message, calling `write_submission_file` on its own does **not** make a submission to the competition.  It merely tells the module to write the `submission.csv` file as part of the Kernel's output.  To make a submission to the competition, you'll have to **Commit** your Kernel and find the generated `submission.csv` file in that Kernel Version's Output tab (note this is _outside_ of the Kernel Editor), then click "Submit to Competition".  When we re-run your Kernel during Stage Two, we will run the Kernel Version (generated when you hit "Commit") linked to your chosen Submission.

## Restart the Kernel to run your code again
In order to combat cheating, you are only allowed to call `make_env` or iterate through `get_prediction_days` once per Kernel run.  However, while you're iterating on your model it's reasonable to try something out, change the model a bit, and try it again.  Unfortunately, if you try to simply re-run the code, or even refresh the browser page, you'll still be running on the same Kernel execution session you had been running before, and the `twosigmanews` module will still throw errors.  To get around this, you need to explicitly restart your Kernel execution session, which you can do by pressing the Restart button in the Kernel Editor's bottom Console tab:
![Restart button](https://i.imgur.com/hudu8jF.png)